In [1]:
from surface_dynamics.databases.flat_surfaces import CylinderDiagrams
from surface_dynamics import AbelianStratum
from utils import valid_equivalence_classes

In [2]:
C = CylinderDiagrams()
H = AbelianStratum(3, 1).components()[0]
# H = AbelianStratum(2, 2).components()[1]
# H = AbelianStratum(2, 1, 1).components()[0]
# H = AbelianStratum(1, 1, 1, 1).components()[0]
# H = AbelianStratum(7, 1).components()[0]

# H = AbelianStratum(2, 2).components()[1]
num_cylinders = 4
num_classes = [2]


diagram_list = list(C.get_iterator(H, num_cylinders))
print(len(diagram_list))

4


In [21]:
class Twist:
    def __init__(self, cd):
        self.cd = cd
        cylinders = cd.cylinders()
        equations = {}
        relations = []

        for bot, top in cylinders:
            row = [0] * cd.degree()
            for s in bot:
                row[s] += 1
            for s in top:
                row[s] -= 1
            relations.append(row)
        relations = matrix(QQ, relations)
        self.relations = relations

        for row in relations.rref():
            for i, one in enumerate(row):
                if one == 1:
                    row[i] = 0
                    for j in range(len(row)):
                        row[j] = -row[j]
                    equations[i] = row
                    break
        self.equations = equations

In [29]:
def check_classes(tw):
    possible = []
    for partition in valid_equivalence_classes(tw.cd, [3]):
        twist_space_vectors = []
        for cl in partition:
            core = vector(QQ, [0] * tw.cd.degree())
            for cyl in cl:
                if cyl in tw.equations:
                    core += tw.equations[cyl]
                else:
                    basis = [0] * tw.cd.degree()
                    basis[cyl] = 1
                    core += vector(QQ, basis)
            twist_space_vectors.append(core)
        V = span(twist_space_vectors, QQ)
        if V.dimension() < 3:
            possible.append(partition)
    return possible

In [30]:
cd = diagram_list[1]
print(cd)
tw = Twist(cd)
print(tw.relations)
check_classes(tw)

(0,3)-(5) (1)-(2) (2,5)-(3,4) (4)-(0,1)
[ 1  0  0  1  0 -1]
[ 0  1 -1  0  0  0]
[ 0  0  1 -1 -1  1]
[-1 -1  0  0  1  0]


[{{0, 3}, {1}, {2}}]

In [31]:
for cd in diagram_list:
    cc = check_classes(Twist(cd))
    if cc:
        print(cd)
        print(cc)

(0,3)-(5) (1)-(2) (2,5)-(3,4) (4)-(0,1)
[{{0, 3}, {1}, {2}}]
(0,1)-(0,2) (2)-(3) (3,4)-(1,5) (5)-(4)
[{{0, 3}, {1}, {2}}, {{0, 2}, {1}, {3}}, {{0, 1}, {2}, {3}}]
